# 1.4 Train-Test Split

### Scaling, folding, evaluating, splitting, training, predicting, finding metric for model.

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
df = pd.read_csv("/Users/theodorsjetnanutvik/Desktop/MaskinLæring/assignment1/data/WineQT.csv")

# Features
X_alcohol = df[["alcohol"]].values
X_chlorides = df[["chlorides"]].values
y = df["quality"].values

# Scaling
scaler = StandardScaler()

# K-Fold Cross Validation (5 splits, shuffle for randomness)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

def evaluate_model(X, y):
    mse_scores, rmse_scores, r2_scores = [], [], [] # 3 empty lists to store metrics for each fold
    
    for train_idx, test_idx in kf.split(X): # For each fold in K-Fold generate train and test indices
        # Split
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # Scale
        X_train_scaled = scaler.fit_transform(X_train) # Learn scaling from train set
        X_test_scaled = scaler.transform(X_test) # Apply same scaling to test set, to avoid data leakage
        
        # Train model
        sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, eta0=0.01, random_state=42) # Stochastic Gradient Descent Regressor
        sgd_reg.fit(X_train_scaled, y_train) # Fit model to training data
        
        # Predict
        y_pred = sgd_reg.predict(X_test_scaled) # Predict on unseen test data using trained model
        
        # Metrics
        mse = mean_squared_error(y_test, y_pred) # Mean Squared Error
        rmse = np.sqrt(mse) # Root Mean Squared Error
        r2 = r2_score(y_test, y_pred) # R-squared
        
        # Saving metrics for this fold
        mse_scores.append(mse) 
        rmse_scores.append(rmse)
        r2_scores.append(r2)
    
    return mse_scores, rmse_scores, r2_scores # Return lists with metrics across folds

# Evaluate for alcohol
mse_a, rmse_a, r2_a = evaluate_model(X_alcohol, y)

# Evaluate for chlorides
mse_c, rmse_c, r2_c = evaluate_model(X_chlorides, y)

print("Alcohol:  MSE =", np.mean(mse_a), "±", np.var(mse_a),
      "| RMSE =", np.mean(rmse_a), "| R2 =", np.mean(r2_a))
print("Chlorides: MSE =", np.mean(mse_c), "±", np.var(mse_c),
      "| RMSE =", np.mean(rmse_c), "| R2 =", np.mean(r2_c))


Alcohol:  MSE = 0.4972592358686095 ± 0.003275680507946046 | RMSE = 0.7040077046357933 | R2 = 0.2335195248005673
Chlorides: MSE = 0.6405079325130248 ± 0.0034978162623649084 | RMSE = 0.7994609519929127 | R2 = 0.0108859466569311


### 1.4.1 How well does alcohol alone predict wine quality in each split?

Using only alcohol as predictor, the model achieves an **RMSE** of approximately $0.70$ and an $R^2$ of $0.23$. This shows alcohol explains some variation in wine quality.

---

### 1.4.2 How well does chloride alone predict wine quality in each split?

With chloride as predictor, the model as an **RMSE** of aobut $0.80$ and an $R^2$ close to $0.01$, indicating chloride levels provide almost no predictive power for wine quality.

---

### 1.4.3 Do you think the model underfits? Why?

With reason we can say that the model underfit. Both models underfit the data because they rely on a single predictor while wine quality depends on multiple features. The low $R^2$ values suggest that the models capture only a small fraction of the true variability.

---

### 1.4.4 

The computed mean and variance of the evaluation metrics across the 5 folds for
both alcohol and chlorides as predictors:

- **Alcohol**  
  - MSE ≈ 0.50 (variance ≈ 0.0033)  
  - RMSE ≈ 0.70  
  - R² ≈ 0.23  

- **Chlorides**  
  - MSE ≈ 0.64 (variance ≈ 0.0035)  
  - RMSE ≈ 0.80  
  - R² ≈ 0.01  

The low variances indicate that the results are consistent across all folds. However,
the alcohol-based model consistently achieves lower error and higher R² than the
chloride-based model, showing that alcohol is a stronger and more reliable predictor
of wine quality.